In [3]:
from pyspark.sql import SparkSession
import os

In [4]:
spark = SparkSession.builder.appName("Spark-Processing-Data").getOrCreate()
sc = spark.sparkContext

your 131072x1 screen size is bogus. expect trouble
24/07/11 13:52:12 WARN Utils: Your hostname, HuyVu resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/07/11 13:52:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/11 13:52:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Laptop

In [62]:
laptop_df = spark.read.csv('./data/raw/tgdd_laptops_data.csv', header=True)
laptop_df.show(truncate=False)

+------------------------------------------------------------+-------+---------+----------+-----------+-------------+-------------+------------+---------+------------------------------------------------------------------------------------+
|name                                                        |brand  |ram      |ssd       |old_price  |current_price|gift         |total_rating|avg_score|link                                                                                |
+------------------------------------------------------------+-------+---------+----------+-----------+-------------+-------------+------------+---------+------------------------------------------------------------------------------------+
|Lenovo Ideapad Slim 3 15IAH8 i5 12450H (83ER000EVN)         |Lenovo |RAM 16 GB|SSD 512 GB|15.990.000₫|14.390.000₫  |Quà1.090.000₫|11          |4.5      |https://www.thegioididong.com/laptop/lenovo-ideapad-slim-3-15iah8-i5-83er00evn      |
|Asus Vivobook Go 15 E1504FA R5 7520U (N

In [63]:
from functions import string_to_int_udf, split_name_product_udf, convert_storage_udf
import pyspark.sql.functions as F

processed_laptop_df= laptop_df.withColumn("result", split_name_product_udf(laptop_df.name))
processed_laptop_df= processed_laptop_df.withColumn("Name", F.col("result.names")) \
                                 .withColumn("Core", F.col("result.cores")) \
                                 .withColumn("Code", F.col("result.codes")) \
                                 .drop("result")
processed_laptop_df= processed_laptop_df.withColumn("RAM", convert_storage_udf(F.col("ram")))
processed_laptop_df= processed_laptop_df.withColumn("SSD", convert_storage_udf(F.col("ssd")))

processed_laptop_df= processed_laptop_df.withColumn("OldPrice", string_to_int_udf(F.col("old_price")))
processed_laptop_df= processed_laptop_df.withColumn("CurrentPrice", string_to_int_udf(F.col("current_price")))
processed_laptop_df= processed_laptop_df.withColumn("GiftValue", string_to_int_udf(F.col("gift")))
processed_laptop_df= processed_laptop_df.withColumnRenamed('brand', 'Brand')\
                                        .withColumnRenamed('total_rating', 'TotalRating')\
                                        .withColumnRenamed('avg_score', 'AverageScore')\
                                        .withColumnRenamed('link', 'LinkProduct')
                                        
processed_laptop_df= processed_laptop_df.withColumn('DiscountPercent', F.round(((F.col("OldPrice") - F.col("CurrentPrice")) / F.col("OldPrice") * 100), 2))
processed_laptop_df = processed_laptop_df.orderBy(['Name', 'CurrentPrice'], ascending = [1, 0])\
                                         .withColumn("ProductType", F.lit('Laptop'))\
                                         .withColumn('Brand', F.when(processed_laptop_df.Brand == 'MacBook', 'Apple').otherwise(processed_laptop_df.Brand))
processed_laptop_df = processed_laptop_df.select(['Name', 'Brand', 'ProductType', 'Core', 'RAM', 'SSD', 'OldPrice', 'CurrentPrice', "DiscountPercent", "GiftValue", 'TotalRating', 'AverageScore', 'LinkProduct'])
processed_laptop_df.show(truncate=False)

+------------------------------------+-----+-----------+--------------+---+---+--------+------------+---------------+---------+-----------+------------+----------------------------------------------------------------------------------------------+
|Name                                |Brand|ProductType|Core          |RAM|SSD|OldPrice|CurrentPrice|DiscountPercent|GiftValue|TotalRating|AverageScore|LinkProduct                                                                                   |
+------------------------------------+-----+-----------+--------------+---+---+--------+------------+---------------+---------+-----------+------------+----------------------------------------------------------------------------------------------+
|Acer Aspire 3 A314 42P              |Acer |Laptop     |R3B3 R7 5700U |16 |512|13990000|13390000    |4.29           |1090000  |0          |0.0         |https://www.thegioididong.com/laptop/acer-aspire-3-a314-42p-r3b3-r7-nxksfsv001                |
|Acer As

### Phone

In [24]:
phone_df = spark.read.json('./data/raw/tgdd_phones_data.json', multiLine=True)
phone_df.show(truncate=False)

+---------+--------------+-------------+-----------+------------------------------------------------------------------------+-------------------------------+-----------+---------------------+------------+
|avg_score|brand         |current_price|gift       |link                                                                    |name                           |old_price  |screen               |total_rating|
+---------+--------------+-------------+-----------+------------------------------------------------------------------------+-------------------------------+-----------+---------------------+------------+
|0.0      |vivo          |5.790.000₫   |Quà100.000₫|https://www.thegioididong.com/dtdd/dtdd/vivo-y28                        |vivo Y28                       |NULL       |6.68 " HD+           |0           |
|4.0      |iPhone (Apple)|29.290.000₫  |NULL       |https://www.thegioididong.com/dtdd/dtdd/iphone-15-pro-max               |iPhone 15 Pro Max              |34.990.000₫|6.7" Super 

In [25]:
from functions import string_to_int_udf
import pyspark.sql.functions as F

processed_phone_df = phone_df.withColumn("OldPrice", string_to_int_udf(F.col("old_price")))
processed_phone_df = processed_phone_df.withColumn("CurrentPrice", string_to_int_udf(F.col("current_price")))\
                                         .withColumn("GiftValue", string_to_int_udf(F.col("gift")))                                        
processed_phone_df = processed_phone_df.withColumnRenamed('screen', 'Screen')\
                                         .withColumnRenamed('brand', 'Brand')\
                                         .withColumnRenamed('total_rating', 'TotalRating')\
                                         .withColumnRenamed('avg_score', 'AverageScore')\
                                         .withColumnRenamed('link', 'LinkProduct')
processed_phone_df = processed_phone_df.withColumn('DiscountPercent', F.round(((F.col("OldPrice") - F.col("CurrentPrice")) / F.col("OldPrice") * 100), 2))
processed_phone_df = processed_phone_df.orderBy(['Name', 'CurrentPrice'], ascending = [1, 0])\
                                       .withColumn('ProductType', F.lit('Phone'))\
                                       .withColumn('Brand', F.when(processed_phone_df.Brand == 'iPhone (Apple)', 'Apple').otherwise(processed_phone_df.Brand))
processed_phone_df = processed_phone_df.select(['Name', 'ProductType', 'Brand', 'Screen', 'OldPrice', 'CurrentPrice', "DiscountPercent", 'GiftValue', 'TotalRating', 'AverageScore', 'LinkProduct'])
processed_phone_df.show(truncate=False)

+-----------------------+-----------+-------+-------------+--------+------------+---------------+---------+-----------+------------+--------------------------------------------------------+
|Name                   |ProductType|Brand  |Screen       |OldPrice|CurrentPrice|DiscountPercent|GiftValue|TotalRating|AverageScore|LinkProduct                                             |
+-----------------------+-----------+-------+-------------+--------+------------+---------------+---------+-----------+------------+--------------------------------------------------------+
|HONOR 90 12GB/256GB    |Phone      |HONOR  |1.5K 6.7"    |NULL    |NULL        |NULL           |NULL     |0          |0.0         |https://www.thegioididong.com/dtdd/dtdd/honor-90        |
|HONOR 90 Lite 8GB/256GB|Phone      |HONOR  |6.7" Full HD+|6490000 |5490000     |15.41          |NULL     |0          |0.0         |https://www.thegioididong.com/dtdd/dtdd/honor-90-lite   |
|HONOR X5 Plus 4GB/64GB |Phone      |HONOR  |6.56"

### Tablet

In [27]:
tablet_df = spark.read.json('./data/raw/tgdd_tablets_data.json', multiLine=True)
tablet_df.show(truncate=False)

+---------+------------+-------------+----+-------------------------------------------------------------------------------------------------+------------------------------------+-----------+--------------------+------------+
|avg_score|brand       |current_price|gift|link                                                                                             |name                                |old_price  |screen              |total_rating|
+---------+------------+-------------+----+-------------------------------------------------------------------------------------------------+------------------------------------+-----------+--------------------+------------+
|0.0      |iPad (Apple)|16.990.000₫  |NULL|https://www.thegioididong.com/may-tinh-bang/may-tinh-bang/ipad-air-m2-11-inch-wifi-128gb         |iPad Air 6 M2 11 inch WiFi          |NULL       |Retina IPS LCD 11"  |0           |
|0.0      |iPad (Apple)|28.990.000₫  |NULL|https://www.thegioididong.com/may-tinh-bang/may-tinh-bang

In [31]:
from functions import string_to_int_udf

processed_tablet_df = tablet_df.withColumn("OldPrice", string_to_int_udf(F.col("old_price")))
processed_tablet_df = processed_tablet_df.withColumn("CurrentPrice", string_to_int_udf(F.col("current_price")))\
                                         .withColumn("GiftValue", string_to_int_udf(F.col("gift")))                                        
processed_tablet_df = processed_tablet_df.withColumnRenamed('screen', 'Screen')\
                                         .withColumnRenamed('brand', 'Brand')\
                                         .withColumnRenamed('total_rating', 'TotalRating')\
                                         .withColumnRenamed('avg_score', 'AverageScore')\
                                         .withColumnRenamed('link', 'ProductLink')
processed_tablet_df = processed_tablet_df.withColumn('DiscountPercent', F.round(((F.col("OldPrice") - F.col("CurrentPrice")) / F.col("OldPrice") * 100), 2))
processed_tablet_df = processed_tablet_df.orderBy(['Name', 'CurrentPrice'], ascending = [1, 0])\
                                         .withColumn('ProductType', F.lit('Tablet'))\
                                         .withColumn('Brand', F.when(processed_tablet_df.Brand == 'iPad (Apple)', 'Apple').otherwise(processed_tablet_df.Brand))
processed_tablet_df = processed_tablet_df.select(['Name', 'ProductType', 'Brand', 'Screen', 'OldPrice', 'CurrentPrice', "DiscountPercent", 'GiftValue', 'TotalRating', 'AverageScore', 'ProductLink'])
processed_tablet_df.show(truncate=False)

+-------------------------------------------+-----------+-------+-----------------------+--------+------------+---------------+---------+-----------+------------+-----------------------------------------------------------------------------------------+
|Name                                       |ProductType|Brand  |Screen                 |OldPrice|CurrentPrice|DiscountPercent|GiftValue|TotalRating|AverageScore|LinkProduct                                                                              |
+-------------------------------------------+-----------+-------+-----------------------+--------+------------+---------------+---------+-----------+------------+-----------------------------------------------------------------------------------------+
|HONOR Pad X9 WiFi 4GB/128GB                |Tablet     |HONOR  |TFT LCD 11.5"          |4990000 |4090000     |18.04          |NULL     |27         |5.0         |https://www.thegioididong.com/may-tinh-bang/may-tinh-bang/honor-pad-x9         

### Brand

In [46]:
laptop_brands_df = processed_laptop_df.select('Brand').distinct()
tablet_brands_df = processed_tablet_df.select('Brand').distinct()
phone_brands_df = processed_phone_df.select('Brand').distinct()

brands = laptop_brands_df.union(tablet_brands_df.union(phone_brands_df)).dropDuplicates().orderBy('Brand')
brands = brands.withColumn('ID', F.monotonically_increasing_id() + 1)
brands = brands.select(["ID", "Brand"])
brands.show()

+---+-------+
| ID|  Brand|
+---+-------+
|  1|   Acer|
|  2|  Apple|
|  3|   Asus|
|  4|   Dell|
|  5|  HONOR|
|  6|     HP|
|  7|   Itel|
|  8| Lenovo|
|  9|    MSI|
| 10|Masstel|
| 11| Mobell|
| 12|  Nokia|
| 13|   OPPO|
| 14|Samsung|
| 15|    TCL|
| 16|  Tecno|
| 17| Xiaomi|
| 18| realme|
| 19|   vivo|
+---+-------+



In [58]:
processed_laptop_df.createOrReplaceTempView("laptops")
processed_tablet_df.createOrReplaceTempView("tablets")
processed_phone_df.createOrReplaceTempView("phones")
brands.createOrReplaceTempView("brands")

### All Products

In [77]:
def retrieve_product_info(product_type):
    query = f""" SELECT 
                    Name, 
                    ProductType, 
                    brands.ID as BrandID, 
                    OldPrice, 
                    CurrentPrice, 
                    DiscountPercent, 
                    GiftValue, 
                    ProductLink
                FROM {product_type} JOIN brands 
                ON {product_type}.Brand = brands.Brand
            """
    product_info = spark.sql(query)
    return product_info

laptops_info = retrieve_product_info('laptops')
tablets_info = retrieve_product_info('tablets')
phones_info = retrieve_product_info('phones')

product_info = laptops_info.union(tablets_info.union(phones_info)).orderBy('Name', ascending=[1])
product_info = product_info.withColumn('ID', F.monotonically_increasing_id() + 1)
product_info = product_info.select(["ID", "Name", "ProductType", "BrandID", "OldPrice", "CurrentPrice", "DiscountPercent", "GiftValue", "ProductLink"])
product_info.show(truncate=False)

+---+------------------------------------+-----------+-------+--------+------------+---------------+---------+----------------------------------------------------------------------------------------------+
|ID |Name                                |ProductType|BrandID|OldPrice|CurrentPrice|DiscountPercent|GiftValue|LinkProduct                                                                                   |
+---+------------------------------------+-----------+-------+--------+------------+---------------+---------+----------------------------------------------------------------------------------------------+
|1  |Acer Aspire 3 A314 42P              |Laptop     |1      |13990000|13390000    |4.29           |1090000  |https://www.thegioididong.com/laptop/acer-aspire-3-a314-42p-r3b3-r7-nxksfsv001                |
|2  |Acer Aspire 3 A315 44P              |Laptop     |1      |12990000|12490000    |3.85           |1090000  |https://www.thegioididong.com/laptop/acer-aspire-a315-44p-r9w8-r7-

In [75]:
def retrieve_product_spec(product_type):
    if product_type == 'laptops':
        query = f"""SELECT 
                       Name, Core, RAM, SSD, 'NULL' AS Screen
                    FROM {product_type}
                """
    else:
        query = f"""SELECT 
                       Name, 'NULL' AS Core, 'NULL' AS RAM, 'NULL' AS SSD, Screen
                    FROM {product_type}
                """
        
    product_spec = spark.sql(query)
    return product_spec

laptops_spec = retrieve_product_spec('laptops')
tablets_spec = retrieve_product_spec('tablets')
phones_spec = retrieve_product_spec('phones')

product_spec = laptops_spec.union(tablets_spec.union(phones_spec)).orderBy("Name", ascending=[1])
product_spec = product_spec.withColumn('ProductID', F.monotonically_increasing_id() + 1)
product_spec = product_spec.select(["ProductID", "Name", "Core", "RAM", "SSD", "Screen"])
product_spec.show(truncate=False)

+---------+------------------------------------+--------------+---+---+------+
|ProductID|Name                                |Core          |RAM|SSD|Screen|
+---------+------------------------------------+--------------+---+---+------+
|1        |Acer Aspire 3 A314 42P              |R3B3 R7 5700U |16 |512|NULL  |
|2        |Acer Aspire 3 A315 44P              |R9W8 R7 5700U |8  |512|NULL  |
|3        |Acer Aspire 3 A315 510P 32EF        |i3 N305       |8  |256|NULL  |
|4        |Acer Aspire 3 A315 58 54XF          |i5 1135G7     |8  |512|NULL  |
|5        |Acer Aspire 3 A315 58 589K          |i5 1135G7     |8  |256|NULL  |
|6        |Acer Aspire 3 A315 59 38PG          |i3 1215U      |8  |512|NULL  |
|7        |Acer Aspire 3 A315 59 5283          |i5 1235U      |8  |512|NULL  |
|8        |Acer Aspire 5 A514 54 5127          |i5 1135G7     |8  |512|NULL  |
|9        |Acer Aspire 5 A514 56P 54TN         |i5 1335U      |8  |512|NULL  |
|10       |Acer Aspire 5 A515 58M 56YX         |i5 1

In [76]:
def retrieve_product_rating(product_type):
    query = f"""SELECT 
                    Name, TotalRating, AverageScore
                FROM {product_type}
               """

    product_rating = spark.sql(query)
    return product_rating

laptops_rating = retrieve_product_rating('laptops')
tablets_rating = retrieve_product_rating('tablets')
phones_rating = retrieve_product_rating('phones')

product_rating = laptops_rating.union(tablets_rating.union(phones_rating)).orderBy("Name", ascending=[1])
product_rating = product_rating.withColumn('ProductID', F.monotonically_increasing_id() + 1)
product_rating = product_rating.select(["ProductID", "Name", "TotalRating", "AverageScore"])
product_rating.show(truncate=False)

+---------+------------------------------------+-----------+------------+
|ProductID|Name                                |TotalRating|AverageScore|
+---------+------------------------------------+-----------+------------+
|1        |Acer Aspire 3 A314 42P              |0          |0.0         |
|2        |Acer Aspire 3 A315 44P              |3          |4.0         |
|3        |Acer Aspire 3 A315 510P 32EF        |26         |4.5         |
|4        |Acer Aspire 3 A315 58 54XF          |45         |4.0         |
|5        |Acer Aspire 3 A315 58 589K          |29         |4.0         |
|6        |Acer Aspire 3 A315 59 38PG          |12         |4.0         |
|7        |Acer Aspire 3 A315 59 5283          |8          |4.0         |
|8        |Acer Aspire 5 A514 54 5127          |25         |4.0         |
|9        |Acer Aspire 5 A514 56P 54TN         |0          |0.0         |
|10       |Acer Aspire 5 A515 58M 56YX         |0          |0.0         |
|11       |Acer Aspire 5 A515 58M 79  

In [34]:
file_path = os.getcwd()
data_path = os.path.join(file_path, 'data/tablets')

if not os.path.exists(data_path):
    os.makedirs(data_path)

tablet_brands_df.write.save(f'{data_path}/tablets_brand.json', format="json")
tablets_info.write.save(f'{data_path}/tablets_info.json', format="json")
tablets_promotion.write.save(f'{data_path}/tablets_promotion.json', format="json")
tablets_price.write.save(f'{data_path}/tablets_price.json', format="json")
tablets_link.write.save(f'{data_path}/tablets_link.json', format="json")